# AOC 2022

Welcome to the Advent of Code 2022 !

## Basic configuration

In [ ]:
!pip install aocd

In [ ]:
import os

os.environ['AOC_SESSION'] = open('session.txt').read().strip()

from aocd import submit
from aocd.models import Puzzle

## Day 4
https://adventofcode.com/2022/day/4
### Part 1

In [ ]:
puzzle = Puzzle(2022, 4)
lines = puzzle.input_data.split('\n')

In [ ]:
ranges = []
for l in lines:
    pair = []
    for r in l.split(','):
        a, b = map(int, r.split('-'))
        pair.append(set(range(a,b+1)))
    ranges.append(pair)

In [ ]:
answer_a = len([r1 for r1, r2 in ranges if (r1 <= r2) or (r2 <= r1)])

In [ ]:
puzzle.answer_a = answer_a

### Part 2

In [ ]:
answer_b = len([r1 for r1, r2 in ranges if r1 & r2])

In [ ]:
puzzle.answer_b = answer_b

## Day 3
https://adventofcode.com/2022/day/3
### Part 1

In [ ]:
puzzle = Puzzle(2022, 3)
lines = puzzle.input_data.split('\n')

In [ ]:
from collections import Counter

sacks = [(Counter(l[:len(l)//2]), Counter(l[len(l)//2:])) for l in lines]

In [ ]:
def prio(c):
    if c.islower():
        return ord(c) - ord('a') + 1
    return ord(c) - ord('A') + 27

answer_a = sum(prio(list(s[0].keys() & s[1].keys())[0]) for s in sacks)

In [ ]:
puzzle.answer_a = answer_a

### Part 2

In [ ]:
sacks = [Counter(l) for l in lines]

answer_b = sum([prio(list(sacks[i].keys() & sacks[i+1].keys() & sacks[i+2].keys())[0]) for i in range(0, len(sacks), 3)])

In [ ]:
puzzle.answer_b = answer_b

## Day 2
https://adventofcode.com/2022/day/2
### Part 1

In [ ]:
puzzle = Puzzle(2022, 2)
lines = puzzle.input_data.split('\n')

In [ ]:
score = 0
for line in lines:
    a, b = line.split()
    a, b = ord(a) - ord('A'), ord(b) - ord('X')
    if a == b:
        score += 3
    if a == (b+1)%3:
        score += 0
    if a == (b+2)%3:
        score += 6
    score += b+1

In [ ]:
puzzle.answer_a = score

### Part 2

In [ ]:
## Short version
score = 0
for line in lines:
    a, b = line.split()
    a, b = ord(a) - ord('A'), ord(b) - ord('X')
    score += 3*b + 1 + (a + (b+2) % 3) % 3

In [ ]:
## Long version
score = 0
for line in lines:
    a, b = line.split()
    a, b = ord(a) - ord('A'), ord(b) - ord('X')
    
    score += 3*(b)

    score +=1
    if b == 0:
        score += (a+2)%3
    elif b==1:
        score += a
    elif b==2:
        score += (a+1)%3

In [ ]:
puzzle.answer_b = score

## Day 1
https://adventofcode.com/2022/day/1
### Part 1

In [ ]:
puzzle = Puzzle(2022, 1)

In [ ]:
lines = puzzle.input_data.split('\n\n')

In [ ]:
answer_a = max([sum(int(e) for e in l.split('\n')) for l in lines])

In [ ]:
puzzle.answer_a = answer_a

### Part 2

In [ ]:
answer_b = sum(sorted([sum(int(e) for e in l.split('\n')) for l in lines], reverse=True)[:3])

In [ ]:
puzzle.answer_b = answer_b